<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Test-a-first-naive-bayesian-model" data-toc-modified-id="Test-a-first-naive-bayesian-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test a first naive bayesian model</a></span></li><li><span><a href="#RF" data-toc-modified-id="RF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>RF</a></span></li><li><span><a href="#XGBOOST" data-toc-modified-id="XGBOOST-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>XGBOOST</a></span></li><li><span><a href="#XGBOOST-FORCE" data-toc-modified-id="XGBOOST-FORCE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>XGBOOST FORCE</a></span><ul class="toc-item"><li><span><a href="#Get-specific-trees" data-toc-modified-id="Get-specific-trees-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Get specific trees</a></span></li></ul></li></ul></div>

## Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter('ignore')
from utils.utils import time_difference_good_format
import pandas as pd
import time
import os
import psutil

## Test a first naive bayesian model

In [ ]:
from models.naive_bayes_k import naive_bayes_k

In [ ]:
k_list = []
begin_time = time.time()
for k in range(1,6):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89', 'sequence']:
            print('Now training: Naive Bayes {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\r')
            test_size, main_class_prop, accuracy = naive_bayes_k(k=k,sequence_origin='DairyDB', 
                                                                 primers_origin='DairyDB', 
                                                                 taxonomy_level=taxonomy_level,
                                                                 selected_primer=selected_primer)
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[k-1], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k)])
    k_df.to_csv('results/models/Naive Bayes - NB({}) - DB - DB - accuracies.csv'.format(k), index=False)

In [ ]:
df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k+1)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/Naive Bayes - ALL NB - DB - DB.csv'.format(k), index=False)

In [ ]:
k_list = []
begin_time = time.time()
for k in range(1,6):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8']:
            print('Now training: Naive Bayes {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\r')
            test_size, main_class_prop, accuracy = naive_bayes_k(k=k,sequence_origin='DairyDB', 
                                                                 primers_origin='Chaudhary', 
                                                                 taxonomy_level=taxonomy_level,
                                                                 selected_primer=selected_primer)
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[k-1], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k)])
    k_df.to_csv('results/models/Naive Bayes - NB({}) - DB - CH - accuracies.csv'.format(k), index=False)

In [ ]:
df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k+1)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/Naive Bayes - ALL NB - DB - CH.csv'.format(k), index=False)

In [ ]:
taxonomy_level = 5
selected_primer = 'V4'

In [ ]:
k_list = []
begin_time = time.time()
all_results = []
for k in range(1,6):
    print('Now training: Naive Bayes {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\r')
    test_size, main_class_prop, accuracy = naive_bayes_k(k=k,sequence_origin='DairyDB', 
                                                         primers_origin='DairyDB', 
                                                         taxonomy_level=taxonomy_level,
                                                         selected_primer=selected_primer)
    all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, k, accuracy])
all_results_df = pd.DataFrame(all_results, columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'k', 'Accuracy'])
all_results_df.to_csv('results/models/Naive Bayes - NB - V4 - Genus - DB - DB - accuracies.csv', index=False)

## RF

In [3]:
from models.random_forest import random_forest_k_default

In [ ]:
test_size, main_class_prop, accuracy = random_forest_k_default(k=5,sequence_origin='DairyDB', 
                                                               primers_origin='Chaudhary', 
                                                               taxonomy_level=5,
                                                               selected_primer='V4', 
                                                               max_depth=25,
                                                               n_estimators=300)

In [4]:
%%time 
RF, test_size, main_class_prop, accuracy = random_forest_k_default(k=4,sequence_origin='DairyDB', 
                                                               primers_origin='Chaudhary', 
                                                               taxonomy_level=5,
                                                               selected_primer='V4', 
                                                               max_depth=25,
                                                               n_estimators=300)

KeyboardInterrupt: 

k_list = []
begin_time = time.time()
for k in range(4,6):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89']:
            test_size, main_class_prop, accuracy = random_forest_k_default(k=k,sequence_origin='', 
                                                                                   primers_origin='DairyDB', 
                                                                                   taxonomy_level=taxonomy_level,
                                                                                   selected_primer=selected_primer)
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[k-4], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Random Forest - RF({})'.format(k)])
    k_df.to_csv('results/models/Random Forest - RF({}) - DB and GG - DB - accuracies.csv'.format(k), index=False)

df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Random Forest - RF({})'.format(k+4)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/Random Forest - RF 4 & 5 - DB and GG - DB.csv'.format(k), index=False)

taxonomy_level = 5
selected_primer = 'V4'

from models.random_forest import random_forest_k_grid_search_cv

In [ ]:
#Number of trees in random forest
n_estimators = [50, 75 , 100, 125, 150]  
# Number of features to consider at every split
max_features = ['auto']  
# Maximum number of levels in tree
max_depth = [None]  
# Minimum number of samples required to split a node
min_samples_split = [2]  
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]  
# Method of selecting samples for training each tree
bootstrap = [False]  
# Create the random grid
param_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth,
    'min_samples_split' : min_samples_split,
    'bootstrap' : bootstrap
}

test_size, main_class_prop, accuracy = random_forest_k_grid_search_cv(k=4,
                                                                      param_grid=param_grid, 
                                                                      sequence_origin='DairyDB', 
                                                                      primers_origin='DairyDB', 
                                                                      taxonomy_level=taxonomy_level,
                                                                      selected_primer=selected_primer)

test_size, main_class_prop, accuracy = random_forest_k_grid_search_cv(k=5,
                                                                      param_grid=param_grid, 
                                                                      sequence_origin='DairyDB', 
                                                                      primers_origin='DairyDB', 
                                                                      taxonomy_level=taxonomy_level,
                                                                      selected_primer=selected_primer)

## XGBOOST

k_list = []
begin_time = time.time()
for k in range(5,0, -1):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89', 'sequence']:
            print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
            process = psutil.Process(os.getpid())
            print('Memory Usage before function: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')
            test_size, main_class_prop, accuracy = xgboost_k_default(k=k,sequence_origin='DairyDB', 
                                                                     primers_origin='DairyDB', 
                                                                     taxonomy_level=taxonomy_level,
                                                                     selected_primer=selected_primer)
            process = psutil.Process(os.getpid())
            print('Memory Usage after function: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[5-k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'XGBoost - XGB({})'.format(k+1)])
    k_df.to_csv('results/models/XGBoost - XGB({}) - DB - DB - accuracies.csv'.format(k), index=False)

df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'XGBoost - XGB({})'.format(5-k)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/XGBoost - ALL XGB - DB - DB.csv', index=False)

## XGBOOST FORCE

In [ ]:
# Number of trees in random forest
n_estimators = [50, 100, 150, 200, 250]  
# Number of features to consider at every split
max_features = ['auto']  
# Maximum number of levels in tree
max_depth = [3, 5, 8, 15]  
# Minimum number of samples required to split a node
min_samples_split = [2]  
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]  
# Method of selecting samples for training each tree
bootstrap = [False]  
# Create the random grid
param_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth,
}

In [ ]:
import warnings
warnings.simplefilter('ignore')
from utils.utils import time_difference_good_format, save_update, restartkernel
import pandas as pd
import time
import os
from models.xgboost import xgboost_k_grid_search_cv
import time
begin_time = time.time()

taxonomy_level=5
selected_primer = 'V4'

k=4
print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(
    k, 
    taxonomy_level, 
    selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
test_size, main_class_prop, accuracy = xgboost_k_grid_search_cv(k=k,
                                                                param_grid=param_grid, 
                                                                sequence_origin='DairyDB', 
                                                                primers_origin='Chaudhary', 
                                                                taxonomy_level=taxonomy_level,
                                                                selected_primer=selected_primer)
save_update('results/models/XGBoost - XGB({}) - V4 - Genus - accuracies.csv'.format(k), k, selected_primer, taxonomy_level, test_size, main_class_prop, accuracy)
    
k=5
print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(
    k, 
    taxonomy_level, 
    selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
test_size, main_class_prop, accuracy = xgboost_k_grid_search_cv(k=k,
                                                                param_grid=param_grid, 
                                                                sequence_origin='DairyDB', 
                                                                primers_origin='Chaudhary', 
                                                                taxonomy_level=taxonomy_level,
                                                                selected_primer=selected_primer)
save_update('results/models/XGBoost - XGB({}) - V4 - Genus - accuracies.csv'.format(k), k, selected_primer, taxonomy_level, test_size, main_class_prop, accuracy)

### Get specific trees

import warnings
warnings.simplefilter('ignore')
from utils.utils import time_difference_good_format, save_update, restartkernel
import pandas as pd
import time
import os
from models.xgboost import xgboost_k_default
import time
import psutil

k=5
taxonomy_level=5
selected_primer='V4'
test_size, main_class_prop, accuracy = xgboost_k_default(k=k,sequence_origin='DairyDB', 
                                                             primers_origin='DairyDB', 
                                                             taxonomy_level=taxonomy_level,
                                                             selected_primer=selected_primer)
save_update('results/models/XGBoost - XGB({}) - DB - DB - accuracies.csv'.format(k), k, selected_primer, taxonomy_level, test_size, main_class_prop, accuracy)
